In [4]:
# Import the necessary libraries
import pandas as pd
from sqlalchemy import create_engine

In [6]:
# Read the flatfiles
file_paths = ["students.csv"]
dataframes = [pd.read_csv(file_path) for file_path in file_paths]
df = pd.concat(dataframes, ignore_index = True)

In [7]:
# Pxreviewing the content of the students csv file.
df.head()

,Name,Age,Grade,Registration Date
0,Jeffery Smith MD,28.0,B,06-08-2024
1,Kristina Lewis,30.0,D,20-10-2021
2,Susan Sloan,NaN,A,01-05-2023
3,Kevin Simpson,NaN,A,13-06-2024
4,Dylan Griffin,NaN,A,01-02-2022


In [10]:
# Transfornmation of datasets

# Renaming the columns
df.columns= df.columns.str.lower().str.replace(" ", "_")


In [11]:
# Ensure Numeric age
df["age"] = pd.to_numeric(df["age"], errors="coerce") 

In [12]:
avg_age = df["age"].mean(skipna=True)

# Replace invalid with average
df["age"] = df["age"].fillna(avg_age).astype(int)

In [13]:
# Conversion of registration to datetime format
df["registration_date"] = pd.to_datetime(df["registration_date"], errors = "coerce")

In [14]:
# Filling of empty registration date gaps making data very consistent
df["registration_date"] = df["registration_date"].fillna(pd.Timestamp("1970-01-01")).dt.strftime("%Y-%m-%d")

In [17]:
# Add a srudent_id column
df["student_id"] = range(1, len(df) + 1)

In [18]:
# Drop rows with missing names
df = df.dropna(subset=["name"])

In [ ]:
import os
from dotenv import load_dotenv
import logging
import traceback
from sqlalchemy import create_engine
import pandas as pd

# Load environment variables from .env file
load_dotenv()

# Set up logging
logging.basicConfig(filename='etl_process.log', level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')

username = os.getenv('MYSQL_USERNAME')
password = os.getenv('MYSQL_PASSWORD')
host = os.getenv('MYSQL_HOST')
database = os.getenv('MYSQL_DATABASE')

try:
    engine = create_engine(f'mysql+pymysql://{username}:{password}@{host}/{database}')
    df.to_sql('students', con=engine, if_exists='replace', index=False)  # Replace 'students' with your table name
    logging.info("ETL Process Completed Successfully!")
    print("ETL Process Completed Successfully!")
except Exception as e:
    logging.error("An error occurred during the ETL process:")
    logging.error(f"Error message: {str(e)}")
    logging.error("Stack trace:")
    logging.error(traceback.format_exc())  # Log detailed stack trace
    print("An error occurred during the ETL process. Check the log for details.")


An error occurred during the ETL process. Check the log for details.
